In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from category_encoders import TargetEncoder

C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
#training_df = pd.read_csv("./tcd ml 2019-20 income prediction training (with labels).csv/tcd ml 2019-20 income prediction training (with labels).csv")
training_df = pd.read_csv("./tcdml1920-income-ind/tcd ml 2019-20 income prediction training (with labels).csv")

In [3]:
#test_df = pd.read_csv("./tcd ml 2019-20 income prediction test (without labels).csv/tcd ml 2019-20 income prediction test (without labels).csv")
test_df = pd.read_csv("./tcdml1920-income-ind/tcd ml 2019-20 income prediction test (without labels).csv")

In [4]:
training_df.shape

(111993, 12)

In [5]:
test_df.shape

(73230, 12)

In [6]:
training_df.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217


In [7]:
test_df.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income
0,111994,1992.0,other,21.0,Honduras,391652,senior project analyst,Master,1,Brown,153,NaN
1,111995,1986.0,other,34.0,Kyrgyzstan,33653,greeter,Bachelor,0,Black,163,NaN
2,111996,1994.0,unknown,53.0,Portugal,34765,liaison,Bachelor,1,Blond,153,NaN
3,111997,1984.0,0,29.0,Uruguay,1494132,occupational therapist,No,0,Black,154,NaN
4,111998,2007.0,other,17.0,Serbia,120661,portfolio manager,No,0,Red,191,NaN


In [8]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111993 entries, 0 to 111992
Data columns (total 12 columns):
Instance             111993 non-null int64
Year of Record       111552 non-null float64
Gender               104561 non-null object
Age                  111499 non-null float64
Country              111993 non-null object
Size of City         111993 non-null int64
Profession           111671 non-null object
University Degree    104623 non-null object
Wears Glasses        111993 non-null int64
Hair Color           104751 non-null object
Body Height [cm]     111993 non-null int64
Income in EUR        111993 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 10.3+ MB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73230 entries, 0 to 73229
Data columns (total 12 columns):
Instance             73230 non-null int64
Year of Record       72935 non-null float64
Gender               68368 non-null object
Age                  72951 non-null float64
Country              73230 non-null object
Size of City         73230 non-null int64
Profession           73035 non-null object
University Degree    68373 non-null object
Wears Glasses        73230 non-null int64
Hair Color           68314 non-null object
Body Height [cm]     73230 non-null int64
Income               0 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 6.7+ MB


In [10]:
categorical_fields = ["Gender", "Country", "Profession", "University Degree"]
numerical_fields = ["Year of Record", "Age", "Size of City", "Body Height [cm]"]
result = ["Income in EUR"]

In [11]:
#Combining the two sets for better encoding (not required for target encoding - splitting later)
training_df["train"] = 1
test_df["train"] = 0

In [12]:
combined = pd.concat([training_df, test_df])

C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [13]:
combined.shape

(185223, 14)

# Feature Transformation

In [14]:
#training_df["Gender"].replace('0', 'unknown', inplace = True)
combined["Gender"].replace('0', 'unknown', inplace = True)

In [15]:
#training_df["University Degree"].replace('0', 'No', inplace=True)
combined["University Degree"].replace('0', 'No', inplace=True)

In [16]:
#training_df.head()
combined.head()

,Age,Body Height [cm],Country,Gender,Hair Color,Income,Income in EUR,Instance,Profession,Size of City,University Degree,Wears Glasses,Year of Record,train
0,41.0,193,Belarus,unknown,Blond,NaN,61031.94416,1,steel workers,1239930,Bachelor,0,1997.0,1
1,41.0,186,Singapore,other,Black,NaN,91001.32764,2,safe event coordinator,1603504,Master,0,1996.0,1
2,28.0,170,Norway,other,Brown,NaN,157982.17670,3,receivables/payables analyst,1298017,PhD,1,2018.0,1
3,33.0,171,Cuba,other,Black,NaN,45993.75793,4,fleet assistant,751903,No,1,2006.0,1
4,46.0,188,United Arab Emirates,female,Blond,NaN,38022.16217,5,lead trainer,95389,No,0,2010.0,1


# Handling Missing Values

In [17]:
for field in categorical_fields:
    #training_df[field].fillna('unknown', inplace=True)
    combined[field].fillna('unknown', inplace=True)

In [18]:
for field in numerical_fields:
    #training_df[field].fillna(training_df[field].mean(), inplace=True)
    combined[field].fillna(combined[field].mean(), inplace=True)

In [19]:
combined.drop(['Wears Glasses', 'Hair Color'], axis=1, inplace=True)
combined.head()

,Age,Body Height [cm],Country,Gender,Income,Income in EUR,Instance,Profession,Size of City,University Degree,Year of Record,train
0,41.0,193,Belarus,unknown,NaN,61031.94416,1,steel workers,1239930,Bachelor,1997.0,1
1,41.0,186,Singapore,other,NaN,91001.32764,2,safe event coordinator,1603504,Master,1996.0,1
2,28.0,170,Norway,other,NaN,157982.17670,3,receivables/payables analyst,1298017,PhD,2018.0,1
3,33.0,171,Cuba,other,NaN,45993.75793,4,fleet assistant,751903,No,2006.0,1
4,46.0,188,United Arab Emirates,female,NaN,38022.16217,5,lead trainer,95389,No,2010.0,1


In [20]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185223 entries, 0 to 73229
Data columns (total 12 columns):
Age                  185223 non-null float64
Body Height [cm]     185223 non-null int64
Country              185223 non-null object
Gender               185223 non-null object
Income               0 non-null float64
Income in EUR        111993 non-null float64
Instance             185223 non-null int64
Profession           185223 non-null object
Size of City         185223 non-null int64
University Degree    185223 non-null object
Year of Record       185223 non-null float64
train                185223 non-null int64
dtypes: float64(4), int64(4), object(4)
memory usage: 18.4+ MB


# MinMax Scaling

In [21]:
scaler = MinMaxScaler()

In [22]:
combined[numerical_fields] = scaler.fit_transform(combined[numerical_fields])

In [23]:
combined.head()

,Age,Body Height [cm],Country,Gender,Income,Income in EUR,Instance,Profession,Size of City,University Degree,Year of Record,train
0,0.241071,0.578947,Belarus,unknown,NaN,61031.94416,1,steel workers,0.024801,Bachelor,0.435897,1
1,0.241071,0.538012,Singapore,other,NaN,91001.32764,2,safe event coordinator,0.032073,Master,0.410256,1
2,0.125000,0.444444,Norway,other,NaN,157982.17670,3,receivables/payables analyst,0.025963,PhD,0.974359,1
3,0.169643,0.450292,Cuba,other,NaN,45993.75793,4,fleet assistant,0.015039,No,0.666667,1
4,0.285714,0.549708,United Arab Emirates,female,NaN,38022.16217,5,lead trainer,0.001907,No,0.769231,1


In [24]:
result_scaler = MinMaxScaler()

In [25]:
training_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
training_df.drop(["train", "Income"], axis=1, inplace=True)
test_df.drop(["train", "Income in EUR"], axis=1, inplace=True)

C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
training_df.shape

(111993, 10)

In [27]:
test_df.shape

(73230, 10)

In [28]:
training_df[result] = result_scaler.fit_transform(training_df[result])

C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [29]:
training_df[result].head()

,Income in EUR
0,0.012612
1,0.018276
2,0.030936
3,0.009770
4,0.008263


In [30]:
training_df.head()

,Age,Body Height [cm],Country,Gender,Income in EUR,Instance,Profession,Size of City,University Degree,Year of Record
0,0.241071,0.578947,Belarus,unknown,0.012612,1,steel workers,0.024801,Bachelor,0.435897
1,0.241071,0.538012,Singapore,other,0.018276,2,safe event coordinator,0.032073,Master,0.410256
2,0.125000,0.444444,Norway,other,0.030936,3,receivables/payables analyst,0.025963,PhD,0.974359
3,0.169643,0.450292,Cuba,other,0.009770,4,fleet assistant,0.015039,No,0.666667
4,0.285714,0.549708,United Arab Emirates,female,0.008263,5,lead trainer,0.001907,No,0.769231


# Splitting data into training and validation sets

In [31]:
training_indices, validation_indices = train_test_split(training_df.index, test_size = 0.20)

In [32]:
X_train = training_df.iloc[training_indices]
X_train.drop(X_train[result], axis=1, inplace=True)
X_validation = training_df.iloc[validation_indices]
X_validation.drop(X_validation[result], axis=1, inplace=True)

C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
y_train = training_df[result].iloc[training_indices]
y_validation = training_df[result].iloc[validation_indices]

In [34]:
X_train.shape

(89594, 9)

In [35]:
X_validation.shape

(22399, 9)

In [36]:
y_train.shape

(89594, 1)

In [37]:
y_validation.shape

(22399, 1)

In [ ]:
# Target Encoding

In [38]:
from category_encoders import TargetEncoder
enc = TargetEncoder(cols=['Gender', 'Country', 'Profession', 'University Degree']).fit(X_train, y_train)

In [39]:
X_train = enc.transform(X_train, y_train)

In [40]:
X_train.head()

,Age,Body Height [cm],Country,Gender,Instance,Profession,Size of City,University Degree,Year of Record
61132,0.178571,0.491228,0.007453,0.022348,61133,0.024079,0.027969,0.020384,0.820513
25292,0.375000,0.678363,0.008070,0.022348,25293,0.025306,0.000996,0.023100,0.333333
46325,0.241071,0.672515,0.013129,0.022348,46326,0.025293,0.055526,0.021731,0.974359
7498,0.285714,0.456140,0.009807,0.022386,7499,0.019511,0.001934,0.020384,0.461538
14111,0.089286,0.438596,0.041207,0.022386,14112,0.026152,0.021018,0.023100,0.179487


In [41]:
X_validation = enc.transform(X_validation, y_validation)

In [43]:
X_validation.head()

,Age,Body Height [cm],Country,Gender,Instance,Profession,Size of City,University Degree,Year of Record
82799,0.116071,0.333333,0.007397,0.022386,82800,0.014598,0.015213,0.021731,0.461538
63874,0.223214,0.631579,0.038522,0.022386,63875,0.015007,0.011167,0.021731,0.948718
101402,0.035714,0.619883,0.005788,0.022348,101403,0.019422,0.018150,0.018672,0.846154
99747,0.133929,0.415205,0.016021,0.022348,99748,0.024079,0.007396,0.021731,0.128205
32126,0.187500,0.590643,0.026583,0.022386,32127,0.035568,0.007872,0.021731,0.641026


# Learning Models

In [44]:
from sklearn.linear_model import LinearRegression

In [45]:
linear_reg_model = LinearRegression()

In [46]:
linear_reg_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [48]:
linear_reg_model.score(X_validation, y_validation)

0.7033758176690325

In [ ]:
# Random Forest

In [49]:
from sklearn.ensemble import RandomForestRegressor

In [50]:
randomforest = RandomForestRegressor()

In [51]:
randomforest.fit(X_train, y_train)

C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\winpython\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\ipykernel\__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [52]:
randomforest.score(X_validation, y_validation)

0.7955287037215513

# Make Predictions

In [53]:
X_test = test_df.drop(["Income"], axis=1)
X_test.shape

(73230, 9)

In [54]:
test_df = enc.transform(X_test)

In [55]:
test_predictions = randomforest.predict(test_df)

In [58]:
test_df["Income"] = result_scaler.inverse_transform(test_predictions.reshape((-1,1)))
#test_df["Income"] = result_scaler.inverse_transform()
test_df["Income"].head()

0    32344.345587
1    12920.268978
2    35568.263980
3    98269.169674
4    30106.588440
Name: Income, dtype: float64

# Submission file

In [59]:
submission_df = pd.read_csv("./tcdml1920-income-ind/tcd ml 2019-20 income prediction submission file.csv")

In [60]:
submission_df = submission_df[["Instance"]].merge(test_df[submission_df.columns], on="Instance")

In [61]:
submission_df.to_csv("submission_final.csv")

# RMSE

In [70]:
from sklearn.metrics import mean_squared_error

In [62]:
y_true = pd.read_csv("./tcdml1920-income-ind/tcd ml 2019-20 income prediction training (with labels).csv")['Income in EUR']

In [69]:
y_validation_true = result_scaler.inverse_transform(y_validation)
y_validation_true

array([[ 21351.08494],
       [171326.3243 ],
       [ 28737.65812],
       ...,
       [ 32651.44744],
       [133100.3518 ],
       [115175.2898 ]])

In [65]:
validation_pred = randomforest.predict(X_validation)
validation_pred = result_scaler.inverse_transform(validation_pred.reshape((-1,1)))
validation_pred

array([[ 23615.38150099],
       [218706.128655  ],
       [ 69811.56271807],
       ...,
       [ 36490.49062567],
       [107566.20322533],
       [ 99751.30541667]])

In [89]:
diff = y_validation_true - validation_pred

In [90]:
sq = diff * diff
sq

array([[5.12703892e+06],
       [2.24484586e+09],
       [1.68706564e+09],
       ...,
       [1.47382526e+07],
       [6.51992743e+08],
       [2.37899294e+08]])

In [91]:
summ = np.sum(sq)
summ

98538292988834.22

In [92]:
rmse = np.sqrt(summ / np.size(validation_indices))
rmse

66326.67134025955